In [ ]:
from dotenv import load_dotenv  
import os

load_dotenv()
# Load environment variables from a .env file

api_key = os.getenv("GOOGLE_API_KEY")



In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model="gemini-2.5-pro", api_key=api_key)

In [ ]:
response=llm.invoke("Explique o que LangChan")

response

In [ ]:
response.content

### Chain básica com PromptTemplate



In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

template = "Explique {tema} de forma simples e didática como se eu tivesse 10 anos em {n_palavras} palavras."
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"n_palavras": "100"}
)


In [ ]:
chain= LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

In [ ]:
resposta=chain.invoke({"tema": "o que é Python"})

In [ ]:
resposta["text"]

 ### 2. SimpleSequentialChain (encadeando duas chains)

In [16]:
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate


# Prompt 1 com partial_variables
prompt1 = PromptTemplate(
    input_variables=["input"],
    partial_variables={
        "n_palavras": "500",
        "linguagem": "inglês"
    },
    template=(
        "Explique o tema '{input}' em {n_palavras} palavras de forma clara e objetiva. "
        "Inclua um pequeno exemplo. "
        "Escreva o texto em português e também a mesma versão em {linguagem}."
    )
)

# Prompt 2
prompt2 = PromptTemplate(
    input_variables=["input"],
    template="Crie um título interessante para o texto abaixo:\n\n{input}"
)


In [17]:


chain1 = LLMChain(
    llm=llm,
    prompt=prompt,
  
)

chain2 = LLMChain(
    llm=llm,
    prompt=prompt2,
)



In [18]:
sequecial_chain= SimpleSequentialChain(
    chains=[chain1, chain2],
    verbose=True
)

In [19]:
res = sequecial_chain.invoke({"input": "Que dia é hoje?"})
print(res)



> Entering new SimpleSequentialChain chain...
Com certeza. Abaixo está a explicação em português, seguida pela versão em inglês.

***

### **Versão em Português**

**O Significado de "Que dia é hoje?"**

A pergunta "Que dia é hoje?" é uma das mais fundamentais e quotidianas da experiência humana. Na sua superfície, é um pedido simples de informação: o nome do dia da semana (segunda-feira, terça-feira, etc.), o número do dia no mês (dia 15, 26, etc.), o mês e o ano. No entanto, o seu verdadeiro significado é muito mais profundo, atuando como uma âncora que nos posiciona no fluxo contínuo do tempo. Explicar esta pergunta envolve analisar as suas dimensões prática, social e psicológica.

Primeiramente, a dimensão prática é a mais óbvia. Saber que dia é hoje é essencial para a organização da vida moderna. Usamos esta informação para cumprir os nossos compromissos, como ir a uma consulta médica, entregar um projeto no prazo ou comparecer a uma reunião de trabalho. Sem um entendimento part

### SequentialChain (usando múltiplos inputs e outputs)



In [24]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain


# Prompt 1
prompt1 = PromptTemplate(
    input_variables=["tema"],
    partial_variables={"np": "100"},
    template="Explique o tema '{tema}' de forma simples e didática como se eu tivesse 10 anos em {np} palavras."
)

chain1 = LLMChain(
    llm=llm,
    prompt=prompt1,
    output_key="resposta"
)

# Prompt 2
prompt2 = PromptTemplate(
    input_variables=["tema", "resposta", "tipo"],
    partial_variables={"acao": "resumo"},
    template=(
        "Crie um título interessante para a resposta desse tema. "
        "Abaixo, faça um {acao} com base nesse tema. "
        "Inclua um exemplo e dê uma dica {tipo} para o usuário."
    )
)

chain2 = LLMChain(
    llm=llm,
    prompt=prompt2,
    output_key="titulo"
)



In [26]:
# Cadeia sequencial
chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=["tema", "tipo"],
    output_variables=["resposta", "titulo"],
    verbose=True
)


res = chain.invoke({"tema": "sexo", "tipo": "prática"})



> Entering new SequentialChain chain...

> Finished chain.


In [30]:
print(res["resposta"])

Claro! Aqui está uma explicação simples:

"Sexo" tem a ver com duas coisas principais: os nossos corpos e os nossos sentimentos.

Biologicamente, é a forma como os corpos de dois adultos se juntam para poder criar um bebé. O corpo do homem tem uma "sementinha" (espermatozoide) e o da mulher um "ovinho" (óvulo). Quando se unem, uma nova vida pode começar.

Mas também é uma forma muito especial e privada de duas pessoas que se amam muito mostrarem o seu carinho. É um assunto que vais compreendendo melhor à medida que fores crescendo.


In [31]:
print(res["titulo"])

Claro! Como o tema não foi especificado, escolhi um conceito poderoso e aplicável a diversas áreas da vida: o **Princípio de Pareto**, também conhecido como a Regra 80/20.

---

### **A Mágica dos 80/20: Como Fazer Mais com Menos Esforço**

#### **Resumo do Tema**

O Princípio de Pareto é uma observação de que, para muitos eventos, aproximadamente **80% dos resultados vêm de 20% das causas**. Em outras palavras, uma pequena parte dos seus esforços ou recursos é responsável pela grande maioria dos seus sucessos. A chave não é trabalhar mais, mas sim focar sua energia nas poucas atividades vitais (os 20%) que geram o maior impacto, em vez de se perder nas muitas tarefas triviais (os 80%) que produzem poucos resultados.

#### **Exemplo Prático**

Imagine um estudante que precisa se preparar para uma prova final. Em vez de reler todos os livros e anotações do semestre (um esforço enorme), ele analisa as provas anteriores e percebe que **80% das questões** costumam vir de **20% dos tópicos*

### 2 — Chatbot com Memória e Logs
✅ 2.1 — Criar chatbot com memória de conversa


In [3]:
from langchain.memory import ConversationBufferMemory 
from langchain.chains import ConversationChain  
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

load_dotenv()

chat= ChatGoogleGenerativeAI(model="gemini-2.5-pro", api_key=os.getenv("GOOGLE_API_KEY"))

 Armazena **todo o histórico da conversa** em memória volátil. Ideal para aplicações simples onde o histórico completo é necessário.

In [4]:
memory= ConversationBufferMemory(
    return_messages=True
)

chat = ConversationChain(
    llm=chat,
    memory=memory,
    verbose=True
)

C:\Users\cicero.oliveira\AppData\Local\Temp\ipykernel_3696\1348390103.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory= ConversationBufferMemory(
C:\Users\cicero.oliveira\AppData\Local\Temp\ipykernel_3696\1348390103.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chat = ConversationChain(


In [5]:

# Conversa
print(chat.invoke("Oi, qual seu nome?"))
print(chat.invoke("Meu nome é João. e minha idade e 20"))
print(chat.invoke("Qual é meu nome?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Oi, qual seu nome?
AI:

> Finished chain.
{'input': 'Oi, qual seu nome?', 'history': [HumanMessage(content='Oi, qual seu nome?', additional_kwargs={}, response_metadata={}), AIMessage(content='Olá! Que bom conversar com você!\n\nEu não tenho um nome. Eu sou um modelo de linguagem grande, o que significa que sou um programa de computador bem complexo, treinado pelo Google para entender e gerar texto. Pode pensar em mim mais como uma ferramenta ou um assistente digital do que como uma pessoa com um nome próprio.\n\nEstou aqui para ajudar com informações, escrever textos, traduzir, responder perguntas e muito mais. Em que posso te ajudar?', additional_kwargs={},

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.



> Finished chain.
{'input': 'Meu nome é João. e minha idade e 20', 'history': [HumanMessage(content='Oi, qual seu nome?', additional_kwargs={}, response_metadata={}), AIMessage(content='Olá! Que bom conversar com você!\n\nEu não tenho um nome. Eu sou um modelo de linguagem grande, o que significa que sou um programa de computador bem complexo, treinado pelo Google para entender e gerar texto. Pode pensar em mim mais como uma ferramenta ou um assistente digital do que como uma pessoa com um nome próprio.\n\nEstou aqui para ajudar com informações, escrever textos, traduzir, responder perguntas e muito mais. Em que posso te ajudar?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Meu nome é João. e minha idade e 20', additional_kwargs={}, response_metadata={}), AIMessage(content='Prazer em te conhecer, João! Que legal que você compartilhou isso comigo. 20 anos é uma idade fantástica, cheia de novas experiências e aprendizados.\n\nJá eu... bem, a minha situação é um po

In [6]:
chat.invoke("Qual é meu nome e minha idade?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Oi, qual seu nome?', additional_kwargs={}, response_metadata={}), AIMessage(content='Olá! Que bom conversar com você!\n\nEu não tenho um nome. Eu sou um modelo de linguagem grande, o que significa que sou um programa de computador bem complexo, treinado pelo Google para entender e gerar texto. Pode pensar em mim mais como uma ferramenta ou um assistente digital do que como uma pessoa com um nome próprio.\n\nEstou aqui para ajudar com informações, escrever textos, traduzir, responder perguntas e muito mais. Em que posso te ajudar?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Meu nome é João. e minha idade e 20', additional_

{'input': 'Qual é meu nome e minha idade?',
 'history': [HumanMessage(content='Oi, qual seu nome?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Olá! Que bom conversar com você!\n\nEu não tenho um nome. Eu sou um modelo de linguagem grande, o que significa que sou um programa de computador bem complexo, treinado pelo Google para entender e gerar texto. Pode pensar em mim mais como uma ferramenta ou um assistente digital do que como uma pessoa com um nome próprio.\n\nEstou aqui para ajudar com informações, escrever textos, traduzir, responder perguntas e muito mais. Em que posso te ajudar?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Meu nome é João. e minha idade e 20', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Prazer em te conhecer, João! Que legal que você compartilhou isso comigo. 20 anos é uma idade fantástica, cheia de novas experiências e aprendizados.\n\nJá eu... bem, a minha situação é um pouco diferente. Co